In [2]:
!pip install pykrx

실습: 삼성전자의 주가 데이터를 가져와서 분석하는 ChatGPT 어플리케이션을 작성하시오(힌트: pykrx 패키지)

In [10]:
import os
import openai
import pandas as pd
from pykrx import stock

openai.api_key = os.getenv("OPENAI_API_KEY")

def analyze_samsung_stock(start_date, end_date):
    """
    삼성전자 주가 데이터를 가져와서 분석 정보를 제공합니다.

    Args:
        start_date (str): 시작 날짜 (YYYYMMDD 형식)
        end_date (str): 종료 날짜 (YYYYMMDD 형식)

    Returns:
        str: 분석 결과 문자열
    """

    try:
        # 삼성전자 티커
        ticker = "005930"

        # 주가 데이터 가져오기
        df = stock.get_market_ohlcv(start_date, end_date, ticker)

        df_string = df.to_string()
        
        prompt = f"""
                다음은 {ticker} 주식의 {start_date}부터 {end_date}까지의 주가 데이터입니다.
        {df_string}
        이 데이터를 분석하고 주요 트렌드와 인사이트를 제공하십시오.
        """
        
        response = openai.chat.completions.create(  # ChatCompletion 사용
            model="gpt-3.5-turbo",  # 또는 다른 적절한 모델
            messages=[
                {"role": "system", "content": "주식 분석 전문가"},  # 시스템 역할 설정 (선택적)
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            n=1,
            stop=None,
            temperature=0.5,
        )
        
        analysis = response.choices[0].message.content

        return analysis

    except Exception as e:
        return f"분석 중 오류가 발생했습니다: {e}"


# 예시 사용
start_date = "20230101"
end_date = "20231231"

# ChatGPT 연동 예시 (가상)
user_query = "2023년 삼성전자 주가 분석해줘"
start_date, end_date = "20230101", "20231231" # 날짜 추출 로직 필요

analysis = analyze_samsung_stock(start_date, end_date)
print(f"사용자 질문: {user_query}\n{analysis}")

사용자 질문: 2023년 삼성전자 주가 분석해줘
주가 데이터를 분석한 결과, 몇 가지 주요 트렌드와 인사이트를 제시할 수 있습니다.

1. 주식 가격 변동성:
   - 2023년 1월부터 12월까지 주식의 가격 변동이 상당히 크다. 등락률이 매일 변동하며, 특히 2023년 4월 7일에는 4.33%의 상승을 기록하였고, 2023년 7월 7일에는 -2.37%의 하락을 보였다.

2. 거래량 변화:
   - 거래량은 주식 시장에서의 활발한 거래를 나타내며, 2023년 9월 1일에 거래량이 29,738,235로 가장 높았다. 이는 주식 시장에서 큰 관심을 받았음을 시사한다.

3. 주가 추이:
   - 주가는 전반적으로 상승하는 추세를 보이고 있으며, 특히 2023년 12월 28일에는 78,500원으로 최고가를 기록하였다.

4. 시장 영향:
   - 주식의 등락률은 주식 시장의 변동에 크게 영향을 받는 것으로 보인다. 특히 급격한 등락이 있을 때는 시장의 변동성이 높은 상황일 수 있다.

5. 시장 전망:
   - 주식의 가격 변동성이 크고 거래량이 활발하며, 주가가 상승하는 추세를 보이고 있으므로, 향후에도 주식 시장에서 긍정적인 흐름이 지속될 수 있다.

이러한 분석을 토대로, 투자자들은 시장의 변동성을 주시하고, 향후 주식의 추이를 주의 깊게 지켜봐야 할 것이며, 투자 결정을 내릴 때에는 신중한 판단이 필요할 것으로 보인다.


실습: 사용자가 입력한 질문에 맞는 캠핑장을 검색해서 추천하는 ChatGPT 어플리케이션을 작성하시오(힌트: 고캠핑 API) 

In [6]:
import os
import openai
import requests
import json

# OpenAI 및 고캠핑 API 키 설정
openai.api_key = os.getenv("OPENAI_API_KEY")
GO_CAMPING_API_KEY = os.getenv("GOCAMPING_API_KEY")
print(GO_CAMPING_API_KEY)

# 고캠핑 API 엔드포인트
GO_CAMPING_API_ENDPOINT = "http://apis.data.go.kr/B551011/GoCamping/basedList"

def recommend_campsites(query):
    """
    사용자 질문에 맞는 캠핑장을 검색하여 추천합니다.

    Args:
        query (str): 사용자 질문

    Returns:
        str: 추천 캠핑장 정보 문자열
    """
    try:
        # API 키 확인
        if not GO_CAMPING_API_KEY:
            return "API 키가 설정되지 않았습니다. 환경 변수 'GOCAMPING_API_KEY'를 확인하세요."

        # API 요청 파라미터 설정
        params = {
            "serviceKey": GO_CAMPING_API_KEY,
            "numOfRows": 10,
            "pageNo": 1,
            "MobileOS": "ETC",
            "MobileApp": "AppTest",
            "_type": "json"
        }

        # API 요청
        response = requests.get(GO_CAMPING_API_ENDPOINT, params=params)

        # 상태 코드 확인
        if response.status_code != 200:
            return f"API 요청 실패: 상태 코드 {response.status_code}, 메시지: {response.text}"

        # 응답 데이터 파싱
        try:
            data = response.json()
        except ValueError as e:
            return f"JSON 디코딩 실패: {e}. 응답: {response.text}"

        # 데이터 구조 확인
        if "response" not in data or "body" not in data["response"] or "items" not in data["response"]["body"]:
            return "캠핑장 데이터가 올바르게 반환되지 않았습니다."

        camps = data['response']['body']['items']['item']

        # 바다 근처 캠핑장 필터링
        camp_info_for_prompt = ""
        for camp in camps:
            if camp.get('lctCl') and ('바다' in camp['lctCl'] or '해변' in camp['lctCl']):
                camp_info_for_prompt += f"캠핑장 이름: {camp['facltNm']}, 위치: {camp['doNm']} {camp['sigunguNm']}, 특징: {camp.get('lineIntro', '정보 없음')}\n"

        if not camp_info_for_prompt:
            return "바다 근처 캠핑장을 찾을 수 없습니다."

        # OpenAI 프롬프트 생성
        prompt = f"""
        사용자는 "{query}"라고 질문했습니다. 다음 캠핑장 정보를 참고하여 바다 근처 캠핑장을 추천해주세요.

        {camp_info_for_prompt}
        """

        # OpenAI API 호출 (gpt-4o-mini 모델)
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # gpt-4o-mini 모델로 변경
            messages=[
                {"role": "system", "content": "캠핑장 추천 전문가"},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0.7,  # 창의적인 답변을 위해 온도 조정
        )

        recommendations = response.choices[0].message.content
        return recommendations

    except requests.exceptions.RequestException as e:
        return f"API 요청 중 오류 발생: {e}"
    except Exception as e:
        return f"추천 중 오류 발생: {e}"


# 예시 사용
user_query = "바다 근처 캠핑장 추천해줘"
recommendations = recommend_campsites(user_query)
print(f"사용자 질문: {user_query}\n{recommendations}")


jstfwuF%2Bvz7jucplIt4R19XUHSb4gtWNWItm8sBrsYCjRSpGfrKj039DxWdNXRJceozhKY7xa4J9%2FAz61aTU%2Bg%3D%3D
사용자 질문: 바다 근처 캠핑장 추천해줘
JSON 디코딩 실패: Expecting value: line 1 column 1 (char 0). 응답: <OpenAPI_ServiceResponse>
	<cmmMsgHeader>
		<errMsg>SERVICE ERROR</errMsg>
		<returnAuthMsg>SERVICE_KEY_IS_NOT_REGISTERED_ERROR</returnAuthMsg>
		<returnReasonCode>30</returnReasonCode>
	</cmmMsgHeader>
</OpenAPI_ServiceResponse>
